### Importing Packages

In [ ]:
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
#import mysql.connector
import pandas as pd
import time
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

### Setting up Spotipy credentials

In [ ]:
client_id= "Client_id"
client_secret= "Client_Secret_Key"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# spotify object to access API
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, retries = 10)

try:
    user = sp.user('Username')
    print('Success!')
    pp.pprint(user)
except:
    print('Connection unsuccessfull')

### Reading the Genres

In [ ]:
genres = pd.read_csv('Genre_List.csv')
genres1 = genres[51:86]
print(len(genres1))
genres1.head()
genres1.loc[genres1["id"] == "Acid-jazz"]

In [ ]:
tracks = pd.read_csv('DM-AudioFeatures.csv')
tracks['genre'] = ""
tracks.head()

### Updating Genres For all Tracks

In [ ]:
def fetchTracks(name):
    finaldataset = pd.DataFrame(columns=['trackid', 'trackname','popularity','releasedate'])
    #finalIdList = []
    #first search
    result = sp.search(q='genre:' + name, type='track', limit = 50)
    #fetch all ids
    finaldataset = finaldataset.append(getdata(result))
    currTracks = result['tracks']
    currnext = result['tracks']['next']
    print("Total Tracks in the Genre " + name + " is : " , result['tracks']['total'])
    i=0
    while(currnext):
        #print(currnext, i)
        #second search
        nextTracks = sp.next(currTracks)
        finaldataset = finaldataset.append(getdata(nextTracks))
        currTracks = nextTracks['tracks']
        currnext = nextTracks['tracks']['next']
        #print(currresult)
        i=i+1
    return finaldataset

In [ ]:
'''def getIds(result):
    currGenreTracks = result['tracks']['items']
    currIds = []
    for i in range(len(currGenreTracks)):
        currIds.append(currGenreTracks[i]['id'])
    return currIds '''

In [ ]:
'''def getdata1(result):
    #dataset = pd.DataFrame(columns=['trackid', 'trackname','popularity','duration'])

    currGenreTracks = result['tracks']['items']
    currtrackIds = []
    trackname = []
    popularity = []
    artistsids = []
    artistsnames = []
    releasedate = []  
    
    
    for i in range(len(currGenreTracks)):
        currtrackIds.append(currGenreTracks[i]['id'])
        trackname.append(currGenreTracks[i]['name'])
        popularity.append(currGenreTracks[i]['popularity'])
        #artists.append(currGenreTracks[i]['album']['artists'])
        releasedate.append(currGenreTracks[i]['album']['release_date'])
        
    dataset = pd.DataFrame({"trackid":currtrackIds,  # Create pandas DataFrame
                        "trackname":trackname,
                        "popularity":popularity,
                        "releasedate":releasedate})
    #print(dataset)
    return dataset '''

In [ ]:
def getdata(result):
    #dataset = pd.DataFrame(columns=['trackid', 'trackname','popularity','releasedate','artists'])

    currGenreTracks = result['tracks']['items']
    currtrackIds = []
    trackname = []
    popularity = []
    artistsids = []
    artistsnames = []
    releasedate = []  
    artists = []
    
    for i in range(len(currGenreTracks)):
        currtrackIds.append(currGenreTracks[i]['id'])
        trackname.append(currGenreTracks[i]['name'])
        popularity.append(currGenreTracks[i]['popularity'])
        getartists(currGenreTracks[i]['artists'])
        artists.append(getartists(currGenreTracks[i]['artists']))
        releasedate.append(currGenreTracks[i]['album']['release_date'])
        
    dataset = pd.DataFrame({"trackid":currtrackIds,  # Create pandas DataFrame
                        "trackname":trackname,
                        "popularity":popularity,
                        "releasedate":releasedate,
                        "artists":artists})
    #print(dataset)
    return dataset

In [ ]:
def getartists(data):
    artists = {}
    for i in range(len(data)):
        artists['id '+str(i+1)] = data[i]['id']
        artists['name '+str(i+1)] = data[i]['name']
        
    #print(artists)
    return artists

In [ ]:
def get_features(dataframe):
    # Define empty lists
    acousticness = []
    danceability = []
    energy = []
    instrumentalness = []
    liveness = []
    loudness = []
    speechiness = []
    tempo = []
    valence = []
    key = []
    mode = []
    trackids = []
    duration = []

    for index, row in dataframe.iterrows():

        # Get track id
        idx = row['trackid']

        # Get corresponding features
        features = sp.audio_features(str(idx))
        trackids.append(idx)
        acousticness.append(features[0]['acousticness'])
        danceability.append(features[0]['danceability'])
        energy.append(features[0]['energy'])
        instrumentalness.append(features[0]['instrumentalness'])
        liveness.append(features[0]['liveness'])
        loudness.append(features[0]['loudness'])
        speechiness.append(features[0]['speechiness'])
        tempo.append(features[0]['tempo'])
        valence.append(features[0]['valence'])
        key.append(features[0]['key'])
        mode.append(features[0]['mode'])
        duration.append(features[0]['duration_ms'])

    # Append the extracted infos to a new dataset

    output = pd.DataFrame(columns=['trackid','acousticness', 'danceability','energy','instrumentalness','liveness','loudness','speechiness','tempo','key','mode','valence','duration'])
    #print(output.head())

    output['trackid'] = trackids
    output['acousticness'] = acousticness
    output['danceability'] = danceability
    output['energy'] = energy
    output['instrumentalness'] = instrumentalness
    output['liveness'] = liveness
    output['loudness'] = loudness
    output['speechiness'] = speechiness
    output['tempo'] = tempo
    output['key'] = key
    output['mode'] = mode
    output['duration'] = duration
    output['valence'] = valence

    return output

In [ ]:
count = 0
dataset = pd.DataFrame(columns=['trackid', 'trackname','popularity','releasedate','artists','acousticness', 'danceability','energy','instrumentalness','liveness','loudness','speechiness','tempo','key','mode','valence','duration'])
    #print(output.head())
for name in genres1['id']:
    
    count = count + 1
    print("Genre Number: ",count)
    print("Genre Name: ", name)
    finaldataset = fetchTracks(name)
    #print(finaldataset)
    finaldataset1 = get_features(finaldataset)
    #print(finaldataset1)
    mergeddataset = pd.merge(finaldataset, finaldataset1, on ='trackid')
    mergeddataset['genre'] = name
    dataset = dataset.append(mergeddataset)
    #print(mergeddataset)
    #for t_id in TrackIds:
    #    tracks.loc[tracks['id'] == t_id, ['genre']] = name

In [ ]:
dataset

In [ ]:
dataset.to_csv(r'D:/DCU/Spotify Assignment/ShevatchaFinalGenre2.csv', index=False, header=True)

In [ ]:
fetchTracks('rock&roll')

In [ ]:
data1 = pd.read_csv("D:/DCU/Spotify Assignment/FinalGenres.csv")
data2 = pd.read_csv("D:/DCU/Spotify Assignment/FinalGenres1.csv")

finaldata = data1.merge(data2,how='outer')

In [ ]:
finaldata.to_csv(r'D:/DCU/Spotify Assignment/FinalGenreData.csv', index=False)

In [ ]:
fetchTracks('Synth-Pop')

In [ ]:
fetchTracks('Teen-pop')

In [ ]:
fetchTracks('Electro-pop')

In [ ]:
fetchTracks('Indie-pop')

In [ ]:
fetchTracks('Bubblegum-pop')

In [ ]:
fetchTracks('Power-pop')

In [ ]:
fetchTracks('Dance-pop')

In [ ]:
fetchTracks('Art-pop')

In [ ]:
fetchTracks('Alternative-Rock')

In [ ]:
fetchTracks('Classic-Rock')

In [ ]:
fetchTracks('Hard-Rock')

In [ ]:
fetchTracks('Soft-Rock')

In [ ]:
fetchTracks('Progressive-Rock')

In [ ]:
fetchTracks('Punk-Rock')

In [ ]:
fetchTracks('Grunge-Rock')

In [ ]:
fetchTracks('Indie-Rock')

In [ ]:
fetchTracks('Old-school hip-hop')

In [ ]:
fetchTracks('Gangsta Rap')

In [ ]:
fetchTracks('Trap')

In [ ]:
fetchTracks('East-Coast hip-hop')

In [ ]:
fetchTracks('West-Coast hip-hop')

In [ ]:
fetchTracks('Southern hip-hop')

In [ ]:
fetchTracks('Conscious hip-hop')

In [ ]:
fetchTracks('Techno')

In [ ]:
fetchTracks('House')

In [ ]:
fetchTracks('Trance')

In [ ]:
fetchTracks('Dubstep')

In [ ]:
fetchTracks('EDM')

In [ ]:
fetchTracks('Ambient')

In [ ]:
fetchTracks('Drum and Bass')

In [ ]:
fetchTracks('Breakbeat')

In [ ]:
fetchTracks('Bluegrass')

In [ ]:
fetchTracks('Country Rock')

In [ ]:
fetchTracks('Western-Swing')

In [ ]:
fetchTracks('Outlaw country')

In [ ]:
fetchTracks('Contemporary country')

In [ ]:
fetchTracks('Cajun')

In [ ]:
fetchTracks('Honky-tonk')

In [ ]:
fetchTracks('Bakersfield-sound')

In [ ]:
fetchTracks('Bebop')

In [ ]:
fetchTracks('Cool jazz')

In [ ]:
fetchTracks('Free jazz')

In [ ]:
fetchTracks('Fusion')

In [ ]:
fetchTracks('Swing')

In [ ]:
fetchTracks('Dixieland')

In [ ]:
fetchTracks('Smooth jazz')

In [ ]:
fetchTracks('Acid-jazz')

In [ ]:
fetchTracks('Funk')

In [ ]:
fetchTracks('Neo-Soul')

In [ ]:
fetchTracks('Motown')

In [ ]:
fetchTracks('Contemporary R&B')

In [ ]:
fetchTracks('Quite-storm')

In [ ]:
fetchTracks('Nothern-soul')

In [ ]:
fetchTracks('Disco')

In [ ]:
fetchTracks('Gospel')

In [ ]:
fetchTracks('Baroque')

In [ ]:
fetchTracks('Romantic')

In [ ]:
fetchTracks('Classical period')

In [ ]:
fetchTracks('Opera')

In [ ]:
fetchTracks('Chamber Music')

In [ ]:
fetchTracks('Choral Music')

In [ ]:
fetchTracks('Symphony')

In [ ]:
fetchTracks('Contemporary Classical')

In [ ]:
fetchTracks('Roots reggae')

In [ ]:
fetchTracks('Dancehall')

In [ ]:
fetchTracks('Dub')

In [ ]:
fetchTracks('Ska')

In [ ]:
fetchTracks('Rocksteady')

In [ ]:
fetchTracks('Ragga')

In [ ]:
fetchTracks('Reggaeton')

In [ ]:
fetchTracks('Lovers Rock')

In [ ]:
fetchTracks('Delta Blues')

In [ ]:
fetchTracks('Chicago Blues')

In [ ]:
fetchTracks('Electric Blues')

In [ ]:
fetchTracks('Jump Blues')

In [ ]:
fetchTracks('Texas Blues')

In [ ]:
fetchTracks('Soul Blues')

In [ ]:
fetchTracks('British Blues')

In [ ]:
fetchTracks('Swamp Blues')

In [ ]:
fetchTracks('Traditional folk')

In [ ]:
fetchTracks('Singer-Songwriter')

In [ ]:
fetchTracks('Americana')

In [ ]:
fetchTracks('Celtic folk')

In [ ]:
fetchTracks('World-folk')

In [ ]:
fetchTracks('Indie folk')

In [ ]:
fetchTracks('Bluegrass')

In [ ]:
fetchTracks('Protest music')

In [ ]:
fetchTracks('Thrash Metal')

In [ ]:
fetchTracks('Death Metal')

In [ ]:
fetchTracks('Heavy Metal')

In [ ]:
fetchTracks('Power Metal')

In [ ]:
fetchTracks('Black Metal')

In [ ]:
fetchTracks('Symphonic-Metal')

In [ ]:
fetchTracks('Doom-Metal')

In [ ]:
fetchTracks('Progressive Metal')

In [ ]:
fetchTracks('Hardcore punk')

In [ ]:
fetchTracks('Pop punk')

In [ ]:
fetchTracks('Skate-punk')

In [ ]:
fetchTracks('Emo')

In [ ]:
fetchTracks('Garage-punk')

In [ ]:
fetchTracks('Anarcho-punk')

In [ ]:
fetchTracks('Post-punk')

In [ ]:
fetchTracks('Riot-grrrl')

In [ ]:
fetchTracks('African')

In [ ]:
fetchTracks('Indian')

In [ ]:
fetchTracks('Latin American')

In [ ]:
fetchTracks('Middle-Eastern')

In [ ]:
fetchTracks('Asian')

In [ ]:
fetchTracks('Celtic')

In [ ]:
fetchTracks('Gypsy')

In [ ]:
fetchTracks('Indigenous')

In [ ]:
fetchTracks('P-funk')

In [ ]:
fetchTracks('Jazz-funk')

In [ ]:
fetchTracks('Disco')

In [ ]:
fetchTracks('G-Funk')

In [ ]:
fetchTracks('Acid-Jazz')

In [ ]:
fetchTracks('Boogie')

In [ ]:
fetchTracks('Electro funk')

In [ ]:
fetchTracks('Funk rock')

In [ ]:
fetchTracks('Black Gospel')

In [ ]:
fetchTracks('Southern Gospel')

In [ ]:
fetchTracks('Contemporary Christian')

In [ ]:
fetchTracks('Gospel Blues')

In [ ]:
fetchTracks('Christian Rock')

In [ ]:
fetchTracks('Gospel-Rap')

In [ ]:
fetchTracks('Urban Contemporary Gospel')

In [ ]:
fetchTracks('Gospel-Jazz')

In [ ]:
fetchTracks('Indie rock')

In [ ]:
fetchTracks('Indie Pop')

In [ ]:
fetchTracks('Folk rock')

In [ ]:
fetchTracks('Shoegaze')

In [ ]:
fetchTracks('Dream-pop')

In [ ]:
fetchTracks('Post-rock')

In [ ]:
fetchTracks('Chamber-pop')

In [ ]:
fetchTracks('Math-rock')

In [ ]:
fetchTracks('Salsa')

In [ ]:
fetchTracks('Reggaeton')

In [ ]:
fetchTracks('Bachata')

In [ ]:
fetchTracks('Merengue')

In [ ]:
fetchTracks('Cumbia')

In [ ]:
fetchTracks('Latin Jazz')

In [ ]:
fetchTracks('Tango')

In [ ]:
fetchTracks('Mariachi')

In [ ]:
fetchTracks('Meditation')

In [ ]:
fetchTracks('Nature-Sound')

In [ ]:
fetchTracks('World-fusion')

In [ ]:
fetchTracks('Space Music')

In [ ]:
fetchTracks('Ambient dub')

In [ ]:
fetchTracks('Progressive electronic')

In [ ]:
fetchTracks('Ethereal')

In [ ]:
fetchTracks('Drone')

In [ ]:
fetchTracks('College Rock')

In [ ]:
fetchTracks('Grunge')

In [ ]:
fetchTracks('Shoegaze')

In [ ]:
fetchTracks('Post-punk')

In [ ]:
fetchTracks('Noise rock')

In [ ]:
fetchTracks('Indie rock')

In [ ]:
fetchTracks('Britpop')

In [ ]:
fetchTracks('Emo')

In [ ]:
data1 = pd.read_csv("D:/DCU/Spotify Assignment/ShevatchaFinalGenre.csv")
data2 = pd.read_csv("D:/DCU/Spotify Assignment/ShevatchaFinalGenre1.csv")
data3 = pd.read_csv("D:/DCU/Spotify Assignment/ShevatchaFinalGenre2.csv")
dataFrame = data1.merge(data2,how='outer')


In [ ]:
data4 = data3.merge(dataFrame,how='outer')

In [ ]:
data4.to_csv(r'D:/DCU/Spotify Assignment/ShevatcheGenresFinally.csv', index=False)